# Building Agent với ChatAnthropic

## 🎯 Mục tiêu

Trong notebook này, chúng ta sẽ:
1. **Xây dựng** một Agent cơ bản với ChatAnthropic
2. **Định nghĩa** và cung cấp Tools cho Agent
3. **Chạy** AgentExecutor với verbose mode
4. **Phân tích** luồng suy nghĩ của Agent
5. **Hiểu** cách Agent quyết định sử dụng tools

## 🏗️ Architecture Overview

```
User Query → Agent (Claude) → Tool Selection → Tool Execution → Result Analysis → Final Answer
                ↑                                      ↓
                └── Tool Results ← Tool Response ←─────┘
```

## Setup và Dependencies

In [ ]:
# Import các thư viện cần thiết
import os
from dotenv import load_dotenv
from typing import List, Dict, Any

# LangChain Core
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import Tool
from langchain_core.prompts import PromptTemplate

# LangChain Anthropic
from langchain_anthropic import ChatAnthropic

# Tools
try:
    from langchain_community.tools import DuckDuckGoSearchRun
    DUCKDUCKGO_AVAILABLE = True
except ImportError:
    print("⚠️ DuckDuckGoSearchRun not available. Will use mock search tool.")
    DUCKDUCKGO_AVAILABLE = False

import requests
import json
import math
from datetime import datetime

# Load environment variables
load_dotenv()

print("✅ Dependencies imported successfully")

In [ ]:
# Khởi tạo ChatAnthropic
llm = ChatAnthropic(
    model="claude-3-5-sonnet-20241022",
    temperature=0,  # Đặt 0 để có reasoning consistent
    anthropic_api_key=os.getenv("ANTHROPIC_API_KEY")
)

# Test LLM
test_response = llm.invoke("Hello! Can you help me build an agent?")
print("✅ ChatAnthropic initialized successfully")
print(f"Test response: {test_response.content[:100]}...")

## 1. Định nghĩa Tools cho Agent

In [ ]:
# Tool 1: Calculator
def calculator_function(expression: str) -> str:
    """Calculate mathematical expressions safely"""
    try:
        # Basic safety: only allow certain characters
        allowed_chars = set('0123456789+-*/.() ')
        if not all(c in allowed_chars for c in expression):
            return "Error: Invalid characters in expression"
        
        # Use eval carefully (trong production, nên dùng safer alternatives)
        result = eval(expression)
        return f"Result: {result}"
    except Exception as e:
        return f"Error calculating expression: {str(e)}"

calculator_tool = Tool(
    name="Calculator",
    description="Useful for mathematical calculations. Input should be a mathematical expression.",
    func=calculator_function
)

print("✅ Calculator tool created")
# Test calculator
test_calc = calculator_tool.run("15 + 25 * 2")
print(f"Calculator test: {test_calc}")

In [ ]:
# Tool 2: Date and Time
def datetime_function(query: str) -> str:
    """Get current date and time information"""
    now = datetime.now()
    
    query_lower = query.lower()
    
    if "date" in query_lower:
        return f"Current date: {now.strftime('%Y-%m-%d (%A)')}"
    elif "time" in query_lower:
        return f"Current time: {now.strftime('%H:%M:%S')}"
    elif "year" in query_lower:
        return f"Current year: {now.year}"
    elif "month" in query_lower:
        return f"Current month: {now.strftime('%B')} ({now.month})"
    else:
        return f"Current date and time: {now.strftime('%Y-%m-%d %H:%M:%S (%A)')}"

datetime_tool = Tool(
    name="DateTime",
    description="Get current date and time information. Can answer questions about current date, time, year, month.",
    func=datetime_function
)

print("✅ DateTime tool created")
# Test datetime
test_dt = datetime_tool.run("what is the current date?")
print(f"DateTime test: {test_dt}")

In [ ]:
# Tool 3: Mock Search (fallback nếu DuckDuckGo không available)
def mock_search_function(query: str) -> str:
    """Mock search function với some sample responses"""
    query_lower = query.lower()
    
    # Mock responses cho common queries
    if "weather" in query_lower:
        return "Weather information: Current temperature is around 25°C, partly cloudy. (This is mock data)"
    elif "bitcoin" in query_lower or "btc" in query_lower:
        return "Bitcoin price: Approximately $43,000 - $45,000 USD. (This is mock data)"
    elif "python" in query_lower and "langchain" in query_lower:
        return "LangChain is a framework for developing applications powered by language models. It's written in Python and supports various LLMs."
    elif "vietnam" in query_lower:
        return "Vietnam is a Southeast Asian country with a population of about 98 million people. Capital: Hanoi, Largest city: Ho Chi Minh City."
    else:
        return f"Mock search results for '{query}': This is a simulated search result. In real implementation, this would return actual web search results."

mock_search_tool = Tool(
    name="Search",
    description="Search the internet for information. Useful for finding current information, facts, and news.",
    func=mock_search_function
)

print("✅ Mock Search tool created")

In [ ]:
# Tool 4: DuckDuckGo Search (nếu available)
if DUCKDUCKGO_AVAILABLE:
    try:
        # Initialize DuckDuckGo search
        ddg_search = DuckDuckGoSearchRun()
        
        # Test search
        test_search = ddg_search.run("LangChain framework")
        print("✅ DuckDuckGo search tool ready")
        print(f"Search test result: {test_search[:100]}...")
        
        # Use real search tool
        search_tool = Tool(
            name="Search",
            description="Search the internet for current information, facts, and news using DuckDuckGo.",
            func=ddg_search.run
        )
        
    except Exception as e:
        print(f"⚠️ DuckDuckGo search failed: {e}")
        print("Using mock search instead")
        search_tool = mock_search_tool
else:
    print("Using mock search tool")
    search_tool = mock_search_tool

In [ ]:
# Tool 5: Text Analyzer
def text_analyzer_function(text: str) -> str:
    """Analyze text properties như word count, character count, etc."""
    try:
        # Basic text analysis
        word_count = len(text.split())
        char_count = len(text)
        char_count_no_spaces = len(text.replace(' ', ''))
        sentence_count = len([s for s in text.split('.') if s.strip()])
        
        # Average words per sentence
        avg_words_per_sentence = word_count / sentence_count if sentence_count > 0 else 0
        
        analysis = {
            "word_count": word_count,
            "character_count": char_count,
            "character_count_no_spaces": char_count_no_spaces,
            "sentence_count": sentence_count,
            "average_words_per_sentence": round(avg_words_per_sentence, 1)
        }
        
        result = "Text Analysis Results:\n"
        for key, value in analysis.items():
            result += f"- {key.replace('_', ' ').title()}: {value}\n"
        
        return result
        
    except Exception as e:
        return f"Error analyzing text: {str(e)}"

text_analyzer_tool = Tool(
    name="TextAnalyzer",
    description="Analyze text properties like word count, character count, sentence count, and readability metrics.",
    func=text_analyzer_function
)

print("✅ Text Analyzer tool created")
# Test text analyzer
test_text = "This is a sample text. It has multiple sentences. Let's analyze it!"
test_analysis = text_analyzer_tool.run(test_text)
print(f"Text analysis test:\n{test_analysis}")

In [ ]:
# Tập hợp tất cả tools
tools = [
    calculator_tool,
    datetime_tool,
    search_tool,
    text_analyzer_tool
]

print(f"\n📋 Available Tools for Agent:")
for i, tool in enumerate(tools, 1):
    print(f"{i}. {tool.name}: {tool.description}")

print(f"\n✅ Total {len(tools)} tools ready for agent")

## 2. Tạo ReAct Agent

In [ ]:
# ReAct prompt template
react_prompt = PromptTemplate.from_template("""
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought: {agent_scratchpad}
""")

print("✅ ReAct prompt template created")

In [ ]:
# Tạo ReAct agent
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=react_prompt
)

print("✅ ReAct agent created successfully")
print(f"Agent type: {type(agent)}")

In [ ]:
# Tạo AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,  # Để xem detailed execution steps
    handle_parsing_errors=True,  # Xử lý parsing errors gracefully
    max_iterations=10,  # Giới hạn số iterations
    early_stopping_method="generate"  # Stop khi có final answer
)

print("✅ AgentExecutor created với verbose=True")
print(f"Max iterations: {agent_executor.max_iterations}")
print(f"Available tools: {[tool.name for tool in tools]}")

## 3. Testing Agent với Simple Queries

In [ ]:
# Test 1: Simple calculation
print("🧮 Test 1: Mathematical Calculation")
print("=" * 50)

query1 = "What is 25 * 4 + 15?"
print(f"Query: {query1}\n")

try:
    result1 = agent_executor.invoke({"input": query1})
    print(f"\n🎯 Final Result: {result1['output']}")
except Exception as e:
    print(f"❌ Error: {e}")

print("\n" + "=" * 70 + "\n")

In [ ]:
# Test 2: Current date/time
print("📅 Test 2: Current Date and Time")
print("=" * 50)

query2 = "What day is today?"
print(f"Query: {query2}\n")

try:
    result2 = agent_executor.invoke({"input": query2})
    print(f"\n🎯 Final Result: {result2['output']}")
except Exception as e:
    print(f"❌ Error: {e}")

print("\n" + "=" * 70 + "\n")

In [ ]:
# Test 3: Text analysis
print("📝 Test 3: Text Analysis")
print("=" * 50)

query3 = "Analyze this text: 'LangChain là một framework mạnh mẽ cho việc xây dựng ứng dụng AI. Nó cung cấp nhiều tools hữu ích.'"
print(f"Query: {query3}\n")

try:
    result3 = agent_executor.invoke({"input": query3})
    print(f"\n🎯 Final Result: {result3['output']}")
except Exception as e:
    print(f"❌ Error: {e}")

print("\n" + "=" * 70 + "\n")

## 4. Complex Multi-Step Queries

In [ ]:
# Test 4: Multi-step calculation với search
print("🔍 Test 4: Multi-step Query (Search + Calculation)")
print("=" * 50)

query4 = "Search for information about Python programming language, then tell me how many words are in the search result."
print(f"Query: {query4}\n")

try:
    result4 = agent_executor.invoke({"input": query4})
    print(f"\n🎯 Final Result: {result4['output']}")
except Exception as e:
    print(f"❌ Error: {e}")

print("\n" + "=" * 70 + "\n")

In [ ]:
# Test 5: Complex reasoning với multiple tools
print("🧠 Test 5: Complex Reasoning (Multiple Tools)")
print("=" * 50)

query5 = """I need to know:
1. What year is it now?
2. Calculate how many years have passed since 1995
3. Search for information about what happened in 1995"""

print(f"Query: {query5}\n")

try:
    result5 = agent_executor.invoke({"input": query5})
    print(f"\n🎯 Final Result: {result5['output']}")
except Exception as e:
    print(f"❌ Error: {e}")

print("\n" + "=" * 70 + "\n")

## 5. Analyzing Agent Behavior

In [ ]:
# Tạo function để capture và analyze agent steps
def analyze_agent_execution(query: str, agent_executor: AgentExecutor):
    """Execute query và analyze agent behavior"""
    print(f"🔍 Analyzing Agent Execution for: '{query}'")
    print("=" * 60)
    
    # Track execution steps
    import time
    start_time = time.time()
    
    try:
        # Execute
        result = agent_executor.invoke({"input": query})
        
        execution_time = time.time() - start_time
        
        print(f"\n📊 Execution Analysis:")
        print(f"   ⏱️  Total time: {execution_time:.2f} seconds")
        print(f"   🎯 Final answer: {result['output'][:100]}...")
        
        return result, execution_time
        
    except Exception as e:
        print(f"❌ Execution failed: {str(e)}")
        return None, 0

print("✅ Analysis function ready")

In [ ]:
# Test với challenging query
challenging_query = "What's the square root of 144, and how does that relate to a dozen?"

result, exec_time = analyze_agent_execution(challenging_query, agent_executor)

if result:
    print(f"\n🧠 Agent Reasoning Analysis:")
    print(f"   The agent successfully:")
    print(f"   1️⃣  Used Calculator tool for mathematical computation")
    print(f"   2️⃣  Made connection between mathematical result và real-world concept")
    print(f"   3️⃣  Provided comprehensive answer")

In [ ]:
# Test agent với ambiguous query
ambiguous_query = "I need to know something about time and numbers"

print("🤔 Test: Ambiguous Query Handling")
print("=" * 50)

result_ambiguous, exec_time_ambiguous = analyze_agent_execution(ambiguous_query, agent_executor)

if result_ambiguous:
    print(f"\n💭 Observation: Agent tried to interpret vague request")
    print(f"   Agent behavior với unclear instructions shows reasoning capability")

## 6. Understanding Agent Decision Making

### 🧠 Agent Thought Process Analysis

Từ các tests trên, chúng ta có thể quan sát:

#### **1. Tool Selection Logic**
- Agent **phân tích** query để xác định loại task
- **Chọn tool** phù hợp nhất dựa trên tool descriptions
- **Fallback** sang tools khác nếu cần thiết

#### **2. Reasoning Pattern**
```
Thought → Action → Observation → Thought → ...
```
- **Thought**: Agent suy luận về approach
- **Action**: Quyết định tool cần sử dụng
- **Observation**: Xem xét kết quả từ tool
- **Iteration**: Lặp lại nếu cần more information

#### **3. Error Recovery**
- Agent có thể **retry** với different inputs
- **Switch tools** khi một tool không hoạt động
- **Adapt strategy** based on intermediate results

In [ ]:
# Test error handling
print("🚨 Test: Error Handling")
print("=" * 50)

error_query = "Calculate 10 divided by zero, then tell me what to do next"
print(f"Query: {error_query}\n")

try:
    result_error = agent_executor.invoke({"input": error_query})
    print(f"\n🎯 Final Result: {result_error['output']}")
    print(f"\n💡 Observation: Agent handled mathematical error gracefully")
except Exception as e:
    print(f"❌ Error: {e}")
    print(f"💡 Observation: Agent execution failed - need better error handling")

print("\n" + "=" * 70 + "\n")

## 7. Custom Tool Integration

In [ ]:
# Tạo custom tool cho Vietnamese text processing
def vietnamese_text_processor(text: str) -> str:
    """Process Vietnamese text và provide insights"""
    try:
        # Basic Vietnamese text analysis
        words = text.split()
        sentences = [s.strip() for s in text.split('.') if s.strip()]
        
        # Count common Vietnamese words
        common_vn_words = ['là', 'của', 'và', 'có', 'trong', 'được', 'với', 'để', 'từ', 'không']
        common_count = sum(1 for word in words if word.lower() in common_vn_words)
        
        # Detect tone marks (Vietnamese specific)
        tone_chars = 'áàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệíìỉĩịóòỏõọôốồổỗộơớờởỡợúùủũụưứừửữựýỳỷỹỵđ'
        tone_count = sum(1 for char in text.lower() if char in tone_chars)
        
        analysis = f"""
Vietnamese Text Analysis:
- Total words: {len(words)}
- Sentences: {len(sentences)}
- Common Vietnamese words: {common_count}
- Characters with tone marks: {tone_count}
- Estimated Vietnamese content: {(tone_count / len(text) * 100):.1f}%
"""
        return analysis.strip()
        
    except Exception as e:
        return f"Error processing Vietnamese text: {str(e)}"

# Add Vietnamese tool to agent
vietnamese_tool = Tool(
    name="VietnameseAnalyzer",
    description="Analyze Vietnamese text for language-specific features như tone marks, common words, and structure.",
    func=vietnamese_text_processor
)

# Update tools list
enhanced_tools = tools + [vietnamese_tool]

print("✅ Vietnamese analyzer tool added")
print(f"Total tools: {len(enhanced_tools)}")

In [ ]:
# Create enhanced agent với new tool
enhanced_agent = create_react_agent(
    llm=llm,
    tools=enhanced_tools,
    prompt=react_prompt
)

enhanced_agent_executor = AgentExecutor(
    agent=enhanced_agent,
    tools=enhanced_tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=10
)

print("✅ Enhanced agent với Vietnamese tool created")

In [ ]:
# Test Vietnamese text analysis
print("🇻🇳 Test: Vietnamese Text Analysis")
print("=" * 50)

vietnamese_query = "Analyze this Vietnamese text: 'Xin chào! Tôi là một trợ lý AI thông minh. Tôi có thể giúp bạn giải quyết nhiều vấn đề khác nhau.'"
print(f"Query: {vietnamese_query}\n")

try:
    vn_result = enhanced_agent_executor.invoke({"input": vietnamese_query})
    print(f"\n🎯 Final Result: {vn_result['output']}")
except Exception as e:
    print(f"❌ Error: {e}")

print("\n" + "=" * 70 + "\n")

## 8. Performance và Optimization Analysis

In [ ]:
# Performance benchmarking
import time

def benchmark_agent(queries: list, agent_executor: AgentExecutor):
    """Benchmark agent performance với multiple queries"""
    results = []
    
    print("🏃 Agent Performance Benchmark")
    print("=" * 50)
    
    for i, query in enumerate(queries, 1):
        print(f"\n{i}. Query: {query[:50]}...")
        
        start_time = time.time()
        try:
            result = agent_executor.invoke({"input": query})
            execution_time = time.time() - start_time
            success = True
            answer_length = len(result['output'])
        except Exception as e:
            execution_time = time.time() - start_time
            success = False
            answer_length = 0
            print(f"   ❌ Failed: {str(e)[:50]}...")
        
        results.append({
            'query': query,
            'execution_time': execution_time,
            'success': success,
            'answer_length': answer_length
        })
        
        print(f"   ⏱️ Time: {execution_time:.2f}s, Success: {'✅' if success else '❌'}")
    
    return results

# Benchmark queries
benchmark_queries = [
    "What is 15 + 27?",
    "What day is today?",
    "Search for LangChain information",
    "Calculate the square root of 64 and tell me what it means",
    "Analyze the text: 'AI is revolutionizing technology'"
]

benchmark_results = benchmark_agent(benchmark_queries, agent_executor)

In [ ]:
# Analyze benchmark results
successful_results = [r for r in benchmark_results if r['success']]
failed_results = [r for r in benchmark_results if not r['success']]

if successful_results:
    avg_time = sum(r['execution_time'] for r in successful_results) / len(successful_results)
    avg_answer_length = sum(r['answer_length'] for r in successful_results) / len(successful_results)
    
    print(f"\n📊 Benchmark Summary:")
    print(f"   ✅ Successful queries: {len(successful_results)}/{len(benchmark_results)}")
    print(f"   ⏱️ Average execution time: {avg_time:.2f} seconds")
    print(f"   📝 Average answer length: {avg_answer_length:.0f} characters")
    print(f"   ❌ Failed queries: {len(failed_results)}")
    
    if failed_results:
        print(f"\n🚨 Failed Queries:")
        for fail in failed_results:
            print(f"   - {fail['query'][:50]}...")
else:
    print("\n❌ No successful executions to analyze")

## 9. Agent Limitations và Improvements

### 🔍 Observations từ Testing

#### **✅ Agent Strengths:**
1. **Tool Selection**: Agent good at choosing appropriate tools
2. **Multi-step Reasoning**: Can break down complex problems
3. **Error Recovery**: Handles some errors gracefully
4. **Adaptability**: Adjusts approach based on intermediate results

#### **⚠️ Areas for Improvement:**
1. **Cost Efficiency**: Multiple LLM calls increase costs
2. **Speed**: Tool selection và reasoning add latency
3. **Reliability**: Non-deterministic behavior
4. **Error Handling**: Some edge cases not handled well

#### **🚀 Optimization Strategies:**
1. **Tool Optimization**: 
   - Better tool descriptions
   - Faster tool implementations
   - Tool result caching

2. **Prompt Engineering**:
   - More specific instructions
   - Better examples
   - Clear success criteria

3. **Execution Control**:
   - Lower max_iterations for simple tasks
   - Timeout configurations
   - Priority-based tool selection

In [ ]:
# Improved agent configuration
def create_optimized_agent(tools, max_iterations=5):
    """Create optimized agent với better configurations"""
    
    # Optimized prompt với clearer instructions
    optimized_prompt = PromptTemplate.from_template("""
You are a helpful assistant with access to specific tools. Answer questions efficiently using the minimum number of tool calls needed.

Available tools:
{tools}

Instructions:
1. Think carefully about which tool(s) you need BEFORE acting
2. Use tools only when necessary - don't use tools for information you already know
3. Be concise in your reasoning
4. Provide clear, direct answers

Format:
Question: the input question you must answer
Thought: think about what you need to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (repeat if needed)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Question: {input}
Thought: {agent_scratchpad}
""")
    
    # Create optimized agent
    optimized_agent = create_react_agent(
        llm=llm,
        tools=tools,
        prompt=optimized_prompt
    )
    
    # Create optimized executor
    optimized_executor = AgentExecutor(
        agent=optimized_agent,
        tools=tools,
        verbose=True,
        handle_parsing_errors=True,
        max_iterations=max_iterations,  # Reduced iterations
        early_stopping_method="generate"
    )
    
    return optimized_executor

# Create optimized agent
optimized_agent_executor = create_optimized_agent(tools, max_iterations=5)

print("✅ Optimized agent created với:")
print(f"   - Reduced max iterations: 5")
print(f"   - Clearer instructions")
print(f"   - Efficiency focus")

In [ ]:
# Compare original vs optimized agent
comparison_query = "What is 50 divided by 2, and what day is today?"

print("⚖️ Comparison: Original vs Optimized Agent")
print("=" * 60)
print(f"Query: {comparison_query}\n")

# Test optimized agent
print("🚀 Optimized Agent:")
print("-" * 30)
start_time = time.time()
try:
    optimized_result = optimized_agent_executor.invoke({"input": comparison_query})
    optimized_time = time.time() - start_time
    print(f"\n⏱️ Optimized execution time: {optimized_time:.2f}s")
    print(f"🎯 Result: {optimized_result['output']}")
except Exception as e:
    print(f"❌ Optimized agent failed: {e}")

print("\n" + "=" * 70)

## 10. Best Practices Summary

### 🎯 Agent Development Best Practices

#### **1. 🔧 Tool Design**
- **Clear descriptions**: Help agent understand when to use each tool
- **Error handling**: Tools should fail gracefully
- **Input validation**: Sanitize inputs before processing
- **Consistent output**: Standardize response formats

#### **2. 🧠 Prompt Engineering**
- **Specific instructions**: Clear guidelines for tool usage
- **Examples**: Show expected behavior patterns
- **Constraints**: Set boundaries for agent actions
- **Format consistency**: Maintain structured reasoning format

#### **3. ⚙️ Configuration Optimization**
- **Max iterations**: Set reasonable limits
- **Temperature**: Use 0 for consistent reasoning
- **Error handling**: Enable graceful failure recovery
- **Verbose mode**: Use for debugging và understanding

#### **4. 🚀 Performance Optimization**
- **Tool caching**: Cache expensive tool results
- **Early stopping**: Stop when answer is found
- **Parallel tools**: Use tools that can run concurrently
- **Monitoring**: Track execution times và success rates

#### **5. 🛡️ Production Considerations**
- **Security**: Validate all inputs và outputs
- **Rate limiting**: Control API usage
- **Logging**: Comprehensive execution logs
- **Fallbacks**: Alternative approaches for failures

### 📊 Key Metrics to Monitor
- **Success rate**: Percentage of queries answered correctly
- **Execution time**: Average time per query
- **Tool usage**: Which tools are used most frequently
- **Error patterns**: Common failure modes
- **Cost efficiency**: Tokens used per successful query

### 🔮 Next Steps
1. **Custom Tools**: Develop domain-specific tools
2. **Memory Integration**: Add conversation memory
3. **Multi-agent Systems**: Coordinate multiple agents
4. **Production Deployment**: Scale và monitor in production
5. **Advanced Patterns**: Implement specialized agent patterns

In [ ]:
# Final summary
print("🎉 Agent Building Tutorial Complete!")
print("=" * 50)
print("\n✅ What we accomplished:")
print("   1. Built basic ReAct agent với ChatAnthropic")
print("   2. Created multiple useful tools")
print("   3. Tested agent với various query types")
print("   4. Analyzed agent reasoning patterns")
print("   5. Optimized agent performance")
print("   6. Learned best practices")

print("\n🎯 Key Takeaways:")
print("   • Agents are powerful but need careful design")
print("   • Tool quality directly impacts agent performance")
print("   • Verbose mode is essential for understanding")
print("   • Optimization balances capability với efficiency")
print("   • Production deployment requires additional considerations")

print("\n🚀 Ready for more advanced agent patterns!")